In [1]:
import pandas as pd
from unicodedata import normalize

#### Loading the lexicon dictionaries and jointing 

In [7]:
# OpLexicon https://www.inf.pucrs.br/linatural/wordpress/recursos-e-ferramentas/oplexicon/
    #https://www.researchgate.net/publication/262175717_Sentiment_Analysis_on_Twitter_Data_for_Portuguese_Language
    #https://aclanthology.org/W11-4507.pdf
lexico = pd.read_csv('dictionaries/lexico_v3.0.txt', sep=",", names=['word', 'PoS', 'score', 'ex'])

In [8]:
# Sentilex-PT http://b2find.eudat.eu/dataset/b6bd16c2-a8ab-598f-be41-1e7aeecd60d3
    # https://www.researchgate.net/publication/344644990_SentiLex-PT_Principais_caracteristicas_e_potencialidades
sentilex_lem = open('dictionaries/SentiLex-lem-PT02.txt','r')
sentilex_flex = open('dictionaries/SentiLex-flex-PT02.txt','r')

In [6]:
sentilex_lem.readline()

'abafado.PoS=Adj;TG=HUM:N0;POL:N0=-1;ANOT=JALC\n'

In [5]:
sentilex_flex.readline()

'à-vontade,à-vontade.PoS=N;FLEX=ms;TG=HUM:N0;POL:N0=1;ANOT=MAN\n'

In [9]:
lexico.head()

word   PoS  score ex
0   =[  emot     -1  A
1   =@  emot     -1  A
2   =p  emot     -1  A
3   =P  emot     -1  A
4   =x  emot     -1  A

In [10]:
columns = [
    'word',
    'PoS',
    'score'
]

joint_dics = pd.DataFrame(columns=columns)

In [11]:
for row in lexico.itertuples(index=True, name='Pandas'):
    joint_dics = joint_dics.append({'word':getattr(row, 'word'),
                                    'PoS':getattr(row, 'PoS'), 
                                    'score':getattr(row, 'score')}, ignore_index=True)

In [12]:
joint_dics.head()

word   PoS score
0   =[  emot    -1
1   =@  emot    -1
2   =p  emot    -1
3   =P  emot    -1
4   =x  emot    -1

#### Getting only necessary information in the lexicon dics

In [13]:
for i in sentilex_flex.readlines():
    pos_dot = i.find('.')            # get the position of the dot character 
    word = (i[:pos_dot])          # get words
    pos_word = word.find(',')    # get the position of the comma character
    word1 = (word[pos_word+1:])          # get first word (variation)
    word2 = (word[:pos_word])          # get second word (variation)
    pol_pos = i.find('POL')            # get position from start of string POL
    polarity = (i[pol_pos+7:pol_pos+9]).replace(';','') # get the words polarity
    pos = i.find(';')              # get semicolon position
    PoSp = (i[:pos])               
    pos = PoSp.find('PoS')        
    PoSp = PoSp[pos+4:]         # get word PoS
    joint_dics = joint_dics.append({'word':word1,            
                                    'PoS':PoSp, 
                                    'score':polarity},
                                   ignore_index=True) # updates the dictionary with the first word the polarity
    if(word1!=word2):
         joint_dics = joint_dics.append({'word':word2,
                                    'PoS':PoSp, 
                                    'score':polarity},
                                   ignore_index=True) # updates the dictionary with the second word the polarity

In [14]:
for i in sentilex_lem.readlines():
    pos_dot = i.find('.')            # get the position of the dot character 
    word = (i[:pos_dot])          # get words
    pol_pos = i.find('POL')            # get position from start of string POL
    polarity = (i[pol_pos+7:pol_pos+9]).replace(';','')         # get the words polarity
    pos = i.find(';')              # get semicolon position
    PoSp = (i[:pos])               
    pos = PoSp.find('PoS')
    PoSp = PoSp[pos+4:]         # get word PoS
    joint_dics = joint_dics.append({'word':word,            
                                    'PoS':PoSp, 
                                    'score':polarity},
                                   ignore_index=True) # updates the dictionary with the word the polarity

In [23]:
joint_dics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89425 entries, 0 to 189946
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    89425 non-null  object
 1   PoS     89425 non-null  object
 2   score   89425 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [16]:
joint_dics_backup = joint_dics

#### Processing the lexicon to remove duplicates and doing

In [17]:
joint_dics['PoS'] = joint_dics['PoS'].str.upper()

In [18]:
joint_dics['word'] = joint_dics['word'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [20]:
joint_dics['score'] = pd.to_numeric(joint_dics['score'])

In [22]:
joint_dics = joint_dics.drop_duplicates(subset ="word")

In [24]:
joint_dics.to_csv(r'dictionaries/joint_dictionaties.csv', index = False)

In [25]:
data = pd.read_csv("tweets_processed.csv")

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1000 non-null   int64  
 1   text          1000 non-null   object 
 2   preprocessed  1000 non-null   object 
 3   sentiment     0 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [27]:
joint_dics.loc[joint_dics['word']=='odio'] # testing single word score - HATE

word PoS  score
155285  odio   N     -1

In [28]:
joint_dics.loc[joint_dics['word']=='abusivo'] # testing single word score - ABUSIVE

word  PoS  score
900  abusivo  ADJ     -1

In [29]:
joint_dics.loc[joint_dics['word']=='alegria'] # testing single word score - HAPPINESS

word PoS  score
40097  alegria   N      1

In [30]:
joint_dics.loc[joint_dics['word']=='viajar'] # testing single word score - TRAVEL

word PoS  score
31752  viajar  VB      0

In [31]:
joint_dics.to_csv(r'dictionaries/joint_dictionaties_processed.csv', index = False)

#### Function to calculate the sentiment score,  the setence is splitted then got each word score in the dictionary and done the sum

In [35]:
def score_sentiment(text):
    text = text.lower()
    l_sentiment = []
    for word in text.split():
        if not joint_dics.loc[joint_dics['word']==word].empty:
            l_sentiment.append(int(joint_dics.loc[joint_dics['word']==word].score.sum()))
    score = sum(l_sentiment)
    return score

In [36]:
score_sentiment('eu odeio dias frios') # I hate cold days

-1

In [37]:
score_sentiment('hoje é um dia especial') # today is a special day

1

#### Function to classify all data

In [38]:
def classify(data):
    for index, row in enumerate(data['text']):
        data.at[index, 'sentiment'] =  score_sentiment(data.at[index, 'preprocessed'])

In [39]:
classify(data)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1000 non-null   int64  
 1   text          1000 non-null   object 
 2   preprocessed  1000 non-null   object 
 3   sentiment     1000 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [43]:
data.head()

id                                               text  \
0  1435695806452011008  @projetoskinny não vai com nenhum pq a gente t...   
1  1435695762948726787  @renanmouraglobo Em SP governo só libera em SP...   
2  1435695755805741059  viagem nunca q ia dar certo no meio da pandemi...   
3  1435695709446197252  @FBITricolor até porque os hospitais estão vaz...   
4  1435695680861921280  also foi bem antes da pandemia (acho q o tweet...   

                                        preprocessed  sentiment  
0             não vai com nenhum gente numa pandemia          0  
1  governo libera logo essa acordo furado espera ...         -1  
2  viagem nunca dar certo meio pandemia queria festa          0  
3  até porque hospitais estão vazios pandemia con...          1  
4  also foi bem antes pandemia acho tweet referia...          0

In [42]:
data['sentiment'] = data['sentiment'].astype(int) 

In [44]:
data.to_csv(r'tweets_classified.csv', index = False)

#### Function only to summarize the total of sentence per sentiment

In [47]:
def count_sentiment(data):
    i = 0
    positive = 0
    negative = 0
    neutro = 0
    for index, row in enumerate(data['text']):
        if data.at[index, 'sentiment'] == 0:
            neutro+=1
        elif data.at[index, 'sentiment'] > 0:
            positive+=1
        else:
            negative+=1
            
    print("Positive: ", positive)
    print("Negative: ", negative)
    print("Neutro: ", neutro)

In [48]:
count_sentiment(data)

Positive:  290
Negative:  428
Neutro:  282


In [49]:
data.loc[data['sentiment']>0][['text', 'sentiment']].style.set_properties(subset=['text'], **{'width': '300px'})

In [50]:
data.loc[data['sentiment']==0][['text', 'sentiment']].style.set_properties(subset=['text'], **{'width': '300px'})

In [51]:
data.loc[data['sentiment']<0][['text', 'sentiment']].style.set_properties(subset=['text'], **{'width': '300px'})

### I collected only 1000 tweets in order to use in the fuctions. The better other hand to create a classification and test it, doing the validation could be having a manual classication and doing a comparition between them.